In [ ]:
# 내 구글 드라이브에 연동
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# 필요한 패키지와 모듈을 불러옴
import cv2
import numpy as np
import time
import io
import base64
from IPython.display import HTML

In [ ]:
# Detection 하기 전에 원본 동영상을 Display
video = io.open('gdrive/My Drive/CV/Object Detection_YOLO/video/video/street.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video width="30%" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
             </video>'''.format(encoded.decode('ascii')))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
file_name = 'gdrive/My Drive/CV/Object Detection_YOLO/video/video/street.mp4'  # Detection 할 원본 동영상
min_confidence = 0.5   # detection 으로 인정할 최소 확률(신뢰도) 지정
output_name = 'street_output_video.mp4'  # Detection 된 output 동영상 
elapsed_time = 0       # 총 경과시간 초기화 

In [ ]:
def detectAndDisplay(frame):
    start_time = time.time()
    img = cv2.resize(frame, None, fx=0.9, fy=0.9)
    height, width, channels = img.shape

    # YOLOv3의 Detecting model 3가지(320×320, 416×416, 608×608)
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []      # detection 한 Class id를 저장하는 배열 정의
    confidences = []    # detection 한 Class 의 신뢰도(확률)를 저장하는 배열 정의
    boxes = []          # detection 한 boxing 정보를 저장하는 배열 정의

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)      # detection 한 Class id
            confidence = scores[class_id]     # detection 한 Class 의 신뢰도(확률)
            if confidence > min_confidence:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])             # boxing 정보를 boxes 배열에 저장
                confidences.append(float(confidence))  # 신뢰도(확률)을 confidences 배열에 저장
                class_ids.append(class_id)             # Class id 를 class_ids 배열에 저장
    
    # apply non-max suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)    # 박스안에 박스(노이즈)를 하나로 만들어 준다.
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:    # 노이즈가 제거된 박스만 표시해 준다.   
            x, y, w, h = boxes[i]
            label = "{}: {:.2f}".format(classes[class_ids[i]], confidences[i]*100)    # Class 이름, 신뢰도(확률) 표시 
            print(i, label)
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
            cv2.rectangle(img, (x, y - 20), (x + w, y), color, -1)
            cv2.putText(img, label, (x + 5, y - 5), font, 1, (255, 255, 255), 1)
    process_time = time.time() - start_time
    global elapsed_time
    elapsed_time += process_time   # 총 경과시간 누적
    print("=== A frame took {:.3f} seconds".format(process_time))

    # video 를 disk 에 output 하기 위해 writer 를 초기화한다.
    global writer
    if writer is None and output_name is not None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(output_name, fourcc, 30,
                (img.shape[1], img.shape[0]), True)
        
    # disk 에 frame 을 write 합니다.
    if writer is not None:
        writer.write(img)

In [ ]:
# Load Yolo
net = cv2.dnn.readNet("gdrive/My Drive/CV/Object Detection_YOLO/yolov3.weights", "gdrive/My Drive/CV/Object Detection_YOLO/yolov3.cfg")  
classes = []  # detection 할 Object(Class) list 배열을 정의
with open("gdrive/My Drive/CV/Object Detection_YOLO/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]   # 80개의 Object(class)를 구분할 수 있는 Object의 이름을 classes 배열에 넣어준다.
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))  # Object 마다 컬러를 하나씩 다르게 지정

# 원본 동영상에서 video stream을 읽어온다.
cap = cv2.VideoCapture(file_name)
writer = None
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('--(!) No captured frame -- Break!')
        print("elapsed time {:.3f} seconds".format(elapsed_time))
        break
    detectAndDisplay(frame)

0 car: 99.80
1 bus: 92.04
2 car: 85.26
3 car: 93.64
4 car: 98.69
5 person: 91.41
6 person: 79.44
7 car: 71.02
8 car: 93.01
10 handbag: 54.26
11 handbag: 52.69
=== A frame took 2.033 seconds
0 car: 99.86
1 bus: 90.03
2 car: 84.90
3 car: 95.70
4 car: 99.18
5 person: 95.11
6 person: 86.28
7 car: 67.57
8 car: 95.93
10 handbag: 58.64
=== A frame took 1.838 seconds
0 car: 99.58
1 bus: 86.50
2 car: 92.51
3 car: 97.41
4 car: 99.21
5 person: 86.89
6 person: 82.37
8 bus: 81.09
9 car: 62.38
10 car: 92.06
11 handbag: 61.19
=== A frame took 1.873 seconds
0 car: 99.70
1 bus: 84.70
2 car: 96.38
3 car: 97.75
4 car: 99.60
5 person: 85.65
6 person: 70.29
7 bus: 67.67
8 car: 59.86
9 car: 94.20
10 handbag: 60.92
=== A frame took 1.917 seconds
0 car: 99.75
1 bus: 88.23
2 car: 94.84
3 car: 96.77
4 car: 99.31
5 person: 91.90
6 person: 76.75
7 bus: 84.25
8 car: 94.74
9 handbag: 58.03
=== A frame took 1.857 seconds
0 car: 99.57
1 bus: 90.23
3 car: 94.97
4 car: 96.53
5 car: 99.01
6 person: 89.61
7 person: 88.78